In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/monumentsdataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (5).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (7).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (4).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (1).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (2).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (3).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download.jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (21).jpg
/kaggle/input/monumentsdataset/training_dataset/Portal remains of St.Paul’s College/download (6).jpg
/kaggle/input/monumentsdataset/training_dataset/ECE Building/IMG_20200118_123947_BURST1.jpg
/k

/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (68).jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (32) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (34) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (51) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (69).jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (6).jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (85) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (80) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (76).jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (49) - Copy.jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/images (83).jpg
/kaggle/input/monumentsdataset/training_dataset/SE Cathedral/download (11).jpg
/kaggle/input/monumentsdataset/tr

In [2]:
import tensorflow as tf
assert tf.__version__.startswith('2')

import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
tf.__version__

'2.1.0'

In [4]:
base_dir = "/kaggle/input/monumentsdataset/training_dataset"
print(base_dir)

/kaggle/input/monumentsdataset/training_dataset


In [5]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 1429 images belonging to 15 classes.
Found 347 images belonging to 15 classes.


In [6]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 15))

In [7]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Adil Shah Palace': 0, 'Basilica of Bom Jesus': 1, 'Chapel of St. Catherine': 2, 'Church of Our Lady of the Rosary': 3, 'Church of St. Augustine': 4, 'ECE Building': 5, 'IT Building': 6, 'Largo of St. Francis Xavier': 7, 'Mahadev Temple': 8, 'Portal remains of St.Paul’s College': 9, 'SE Cathedral': 10, 'St. Cajetan Church': 11, 'arch of viceroy': 12, 'chapora fort': 13, 'fort aguada': 14}


In [8]:
!cat labels.txt

Adil Shah Palace
Basilica of Bom Jesus
Chapel of St. Catherine
Church of Our Lady of the Rosary
Church of St. Augustine
ECE Building
IT Building
Largo of St. Francis Xavier
Mahadev Temple
Portal remains of St.Paul’s College
SE Cathedral
St. Cajetan Church
arch of viceroy
chapora fort
fort aguada

In [9]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')
# base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                               include_top=False, 
#                                               weights='imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [10]:
base_model.trainable = False

In [12]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(15, activation='softmax')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 3, 3, 32)          589856    
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 22,393,135
Trainable params: 590,351
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
epochs = 10

history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator))

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
history_fine = model.fit(train_generator, 
                         steps_per_epoch=len(train_generator), 
                         epochs=5, 
                         validation_data=val_generator, 
                         validation_steps=len(val_generator))

In [ ]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)